In [1]:
import pandas as pd

In [2]:
# Task 1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей,
# которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца
# timestamp для данного значения userId.

In [3]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
ratings_1 = ratings.groupby('userId').count()[['movieId']].sort_values('movieId', ascending = False).reset_index()
ratings_1.rename(columns={'movieId': 'number_of_ratings'}, inplace=True)
ratings_1.head()

,userId,number_of_ratings
0,547,2391
1,564,1868
2,624,1735
3,15,1700
4,73,1610


In [5]:
ratings_2 = ratings.groupby('userId').agg([min, max])['timestamp'].reset_index()
ratings_2['average_live_term'] = ratings_2['max'] - ratings_2['min']
ratings_2.head()

,userId,min,max,average_live_term
0,1,1260759108,1260759205,97
1,2,835355395,835356246,851
2,3,1298861589,1298932787,71198
3,4,949778714,949982274,203560
4,5,1163373044,1163375145,2101


In [6]:
result = ratings_1.merge(ratings_2, on = 'userId', how = 'left')
result

,userId,number_of_ratings,min,max,average_live_term
0,547,2391,974777109,1476587644,501810535
1,564,1868,974708446,976224804,1516358
2,624,1735,1019123866,1476616373,457492507
3,15,1700,997937239,1469330735,471393496
4,73,1610,1255500852,1476086345,220585493
...,...,...,...,...,...
666,296,20,1298158632,1298166310,7678
667,289,20,1328935420,1328935859,439
668,249,20,839461218,839463782,2564
669,221,20,1111551492,1111551662,170


In [7]:
result.query('number_of_ratings > 100', inplace=True)
result

,userId,number_of_ratings,min,max,average_live_term
0,547,2391,974777109,1476587644,501810535
1,564,1868,974708446,976224804,1516358
2,624,1735,1019123866,1476616373,457492507
3,15,1700,997937239,1469330735,471393496
4,73,1610,1255500852,1476086345,220585493
...,...,...,...,...,...
253,217,104,1108160000,1108160981,981
254,36,104,847056510,853005800,5949290
255,623,103,1225255417,1225258519,3102
256,257,103,1338006929,1449941803,111934874


In [8]:
# Task 2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия).
# Необходимо сформировать две таблицы:
# таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
# аналогичную таблицу по типам выручки с указанием адреса клиента
# Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [9]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [10]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [11]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [12]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
total_revenue = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
total_revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [14]:
total_rev_with_adress = client_base.merge(total_revenue.fillna(0), on='client_id', how='left')
total_rev_with_adress

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0
